In [1]:
import torch
import numpy as np
x = torch.rand([5, 2, 3, 4]) # N * d0 * d1 * d2
w = torch.rand([24, 3]) # D * M, (D = d0 * d1 * d2)
b = torch.rand([3, ]) # M
print(len(x.shape))
print(torch.sum(w, axis = 0, keepdim = True))

4
tensor([[12.4602, 10.7019, 11.6882]])


In [2]:
N = x.shape[0]

num = len(x.shape)
print(num)
D = 1
D0, M = w.shape
#print(D0)
D_vec = [N]
for i in range(1, num, 1):
    D_vec.append(x.shape[i]) # [d0, d1, d2]

for i in range(1, num, 1):
    D *= x.shape[i]

x_row = x.reshape([N, D]) # N * D
out = x_row.mm(w) + b
out = torch.maximum(torch.zeros_like(out), out)
print(D_vec, (out > 0))


XX = torch.rand([N, D])
dout = torch.rand([N, M])
dReLU = (out > 0) * dout
print(dout, '\n', dReLU)
dx = XX.reshape(D_vec) # N * D --> N * d0 * d1 * d2
db = torch.sum(dout, axis = 0)
print(db)

4
[5, 2, 3, 4] tensor([[True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True],
        [True, True, True]])
tensor([[0.2781, 0.0152, 0.8285],
        [0.9984, 0.7972, 0.7096],
        [0.3702, 0.6674, 0.8093],
        [0.6854, 0.5422, 0.9972],
        [0.2853, 0.2787, 0.2219]]) 
 tensor([[0.2781, 0.0152, 0.8285],
        [0.9984, 0.7972, 0.7096],
        [0.3702, 0.6674, 0.8093],
        [0.6854, 0.5422, 0.9972],
        [0.2853, 0.2787, 0.2219]])
tensor([2.6175, 2.3007, 3.5664])


In [3]:
param = {}
ws = 1e-2
hidden_dims = torch.randperm(4) + 1
input_dim = 5
num_layers = len(hidden_dims) + 1
# g0 = torch.rand(input_dim, hidden_dims[0])
# param['W1'] = (1 / np.sqrt( 2 * np.pi ) * w) * torch.exp(-0.5 * g0 * g0 / (w * w))

print(hidden_dims)
for it in range(num_layers - 1):
    W_name = "W%d" %(it + 1)
    if it == 0:
        g = torch.rand(input_dim, hidden_dims[0])
#     elif it == (num_layers - 1):
#          g = torch.rand(hidden_dims[num_layers - 2], num_classes)
    else:
        g = torch.rand(hidden_dims[it - 1], hidden_dims[it])
    param[W_name] = (1 / np.sqrt( 2 * np.pi ) * ws) * torch.exp(-0.5 * g * g / (ws * ws))
# for it in range(len(hidden_dims) - 1):
#     W_name = "W%d" % (it + 2)
#     # W_name = "W" + str(it + 2)
#     gaussian = torch.rand(hidden_dims[it], hidden_dims[it + 1])
#     param[W_name] = (1 / np.sqrt( 2 * np.pi ) * w) * torch.exp(-0.5 * gaussian * gaussian / (w * w))
#     #print(param[W_name], it)
print(param)

tensor([1, 4, 2, 3])
{'W1': tensor([[0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [4.8596e-08],
        [0.0000e+00]]), 'W2': tensor([[0.0000e+00, 1.0082e-20, 0.0000e+00, 0.0000e+00]]), 'W3': tensor([[3.3340e-24, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00]]), 'W4': tensor([[0.0000, 0.0000, 0.0008],
        [0.0000, 0.0000, 0.0000]])}


In [4]:
print(hidden_dims[-4])
LR_cache = {}

for i in range(5, 0, -1):
    LR_cache[i + 1] = np.exp(i)
print(LR_cache)
print(LR_cache[3])

u = np.random.rand(*b.shape) < 0.5
u0 = torch.rand(*b.shape) < 0.5
print(b * u0)
b * u

tensor(1)
{6: 148.4131591025766, 5: 54.598150033144236, 4: 20.085536923187668, 3: 7.38905609893065, 2: 2.718281828459045}
7.38905609893065
tensor([0.0000, 0.9848, 0.3910])


tensor([0.0000, 0.0000, 0.3910])

In [65]:
np.sqrt(2 * np.pi)

2.5066282746310002

In [19]:
#print(x_row)
#print(x_row.reshape(x.shape))
#print(x_row.view(x.shape))
#print(x_row.view(x.shape) == x_row.reshape(x.shape))
print(x.shape)
x_row = x.view(x.shape[0], -1)
print(x_row.shape)

torch.Size([2, 3, 4])
torch.Size([5, 24])


## Rotate matrix

In [11]:
W_f = torch.rand(3, 4, 2, 2)
print(W_f)
W_f.transpose(2, 3)
print(W_f)

tensor([[[[0.8351, 0.7733],
          [0.9807, 0.3554]],

         [[0.0965, 0.5460],
          [0.4826, 0.4192]],

         [[0.3154, 0.5795],
          [0.2136, 0.8612]],

         [[0.2941, 0.7209],
          [0.7095, 0.9664]]],


        [[[0.5482, 0.6978],
          [0.5921, 0.2559]],

         [[0.2549, 0.2895],
          [0.8023, 0.7819]],

         [[0.7965, 0.9899],
          [0.7454, 0.0882]],

         [[0.3860, 0.3873],
          [0.2498, 0.0396]]],


        [[[0.2459, 0.1550],
          [0.8622, 0.8264]],

         [[0.5140, 0.0087],
          [0.3564, 0.1432]],

         [[0.4434, 0.2489],
          [0.0140, 0.2768]],

         [[0.4537, 0.5329],
          [0.4750, 0.4575]]]])
tensor([[[[0.8351, 0.7733],
          [0.9807, 0.3554]],

         [[0.0965, 0.5460],
          [0.4826, 0.4192]],

         [[0.3154, 0.5795],
          [0.2136, 0.8612]],

         [[0.2941, 0.7209],
          [0.7095, 0.9664]]],


        [[[0.5482, 0.6978],
          [0.5921, 0.2559]],

       

In [28]:
x = torch.zeros(1, 1, 1, 4, 4)
print(x)
for i in range(4):
    x[:, :, :, i] = i
print(x)
x90 = x.transpose(3, 4)
x180 = x.rot90(2, [3, 4])
print(x180)

tensor([[[[[0., 0., 0., 0.],
           [0., 0., 0., 0.],
           [0., 0., 0., 0.],
           [0., 0., 0., 0.]]]]])
tensor([[[[[0., 0., 0., 0.],
           [1., 1., 1., 1.],
           [2., 2., 2., 2.],
           [3., 3., 3., 3.]]]]])
tensor([[[[[3., 3., 3., 3.],
           [2., 2., 2., 2.],
           [1., 1., 1., 1.],
           [0., 0., 0., 0.]]]]])


In [22]:
x = torch.arange(8).view(2,2,2)

print(x)
torch.rot90(x, 1, [1, 2])

in_dim = [1, 3, 7, 7]
cha = torch.randn(in_dim)
print(cha.shape)

tensor([[[0, 1],
         [2, 3]],

        [[4, 5],
         [6, 7]]])
torch.Size([1, 3, 7, 7])


In [30]:

x2 = torch.randn(2,2)
x2_maximum = torch.maximum(torch.zeros_like(x2), x2)
x2_maximum2 = torch.maximum(x2, torch.zeros_like(x2))
print(torch.zeros_like(x2), '\n', x2_maximum2)
print('\n',x2, '\n',x2[x2>0], '\n', x2 * (x2 > 0), '\n', x2_maximum)
print(torch.randn(3, 3, 2, 2).shape)
6 * torch.ones(3,3)
x1 = []
x1.append([1,2,3])
x1.append([4,5,6])
print(x1.pop())



tensor([[0., 0.],
        [0., 0.]]) 
 tensor([[0.0000, 0.0000],
        [1.3316, 0.0000]])

 tensor([[-0.4245, -0.0954],
        [ 1.3316, -0.2383]]) 
 tensor([1.3316]) 
 tensor([[-0.0000, -0.0000],
        [1.3316, -0.0000]]) 
 tensor([[0.0000, 0.0000],
        [1.3316, 0.0000]])
torch.Size([3, 3, 2, 2])
[4, 5, 6]
tensor([[0.4993, 0.2629, 0.2566],
        [0.9670, 0.4593, 0.0375]]) tensor([0.3396, 0.4879])
tensor([0.0191, 0.2166]) tensor([0.0191, 0.2166])


In [32]:
x3 = torch.rand(2,3)
mu = x3.mean(dim=1)
std = x3.std(dim=1)
print(x3, mu)
(x3[1,] - mu[1])/torch.sqrt(std[1]**2)
print(x3.std(dim=1)**2, x3.var(dim=1))
x4 = torch.rand(1,3)
x4 * x3

tensor([[0.6170, 0.5535, 0.9456],
        [0.1557, 0.1038, 0.0147]]) tensor([0.7053, 0.0914])
tensor([0.0443, 0.0051]) tensor([0.0443, 0.0051])


tensor([[0.5462, 0.4324, 0.1227],
        [0.1378, 0.0811, 0.0019]])

In [16]:
x5 = torch.rand(3, 4, 5, 6)
x5_a = torch.rand(2, 4, 5, 6)
y1 = x5[:, :, 2:5, 3:6]
print(x5.shape, '\n',y1.shape)
for i in range(2):
    y2 = x5 * x5_a[i,]
print(y2.shape)


torch.Size([3, 4, 5, 6]) 
 torch.Size([3, 4, 3, 3])
torch.Size([3, 4, 5, 6])
